### Run notebook 02_LoL_predictor_Data_Preparation before running this one, as it prepares the data!

In [17]:
import logging
import numpy as np
import os
import pandas as pd
import wandb
import yaml

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score
import torch
import torch.nn as nn
from tqdm import tqdm

In [18]:
logging.basicConfig(filename="training_log.log",
                    filemode='a',
                    format='%(asctime)s,%(msecs)d %(name)s %(levelname)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

In [19]:
os.environ["WANDB_SILENT"] = "true"   # silence WANDB init as it gets a bit annoying with bigger trainings

with open('config.yml', mode="r") as f:
    config = yaml.safe_load(f)

wandb.login()

True

In [20]:
pd.set_option('display.max_columns', 500)
DATA_FOLDER = '../data/'

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [21]:
def compute_loss(y_hat, y):
    return nn.BCELoss()(y_hat, y)

# Calculate accuracy (a classification metric)
def accuracy_fn(y_true, y_pred):
    correct = torch.eq(y_true, y_pred).sum().item() # torch.eq() calculates where two tensors are equal
    acc = (correct / len(y_pred)) * 100
    return acc

In [22]:
train = pd.read_csv(f"{DATA_FOLDER}/train.csv")
test = pd.read_csv(f"{DATA_FOLDER}/test.csv")
validation = pd.read_csv(f"{DATA_FOLDER}/validation.csv")

In [23]:
def prepare_pytorch_split(df: pd.DataFrame) -> tuple:
    """Split data for prediction and match them to fit PyTorch models"""
    X = df.drop(columns={"blueWins"})
    y = df.blueWins
    
    X = torch.tensor(X.values)
    y = torch.tensor(y.values)
    
    # https://stackoverflow.com/a/60440460/12342419
    y = y.type(torch.LongTensor)
    
    X = X.to(torch.float32).to(device)
    y = y.to(torch.float32).to(device)
    
    y = y.reshape((y.shape[0], 1))

    return(X, y)

In [24]:
wandb.init(
        project="lol-predictor",
        config=config['train'],
        group='pytorch',
        mode='online'
    )

In [25]:
trainX, trainy = prepare_pytorch_split(train)
testX, testy = prepare_pytorch_split(test)
valX, valy = prepare_pytorch_split(validation)

In [26]:
nHidden = config['train']['inputNeurons']

model = nn.Sequential(
    nn.Linear(config["preparation"]["features_amount"], nHidden[0]),
    nn.ReLU(),
    nn.Linear(nHidden[0], nHidden[1]),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(nHidden[1], nHidden[2]),
    nn.ReLU(),
    nn.Linear(nHidden[2], nHidden[3]),
    nn.Dropout(0.5),
    nn.ReLU(),
    nn.Linear(nHidden[3], 1),
    nn.Sigmoid()
    # Softmax
)

In [27]:
model

Sequential(
  (0): Linear(in_features=20, out_features=128, bias=True)
  (1): ReLU()
  (2): Linear(in_features=128, out_features=32, bias=True)
  (3): Dropout(p=0.5, inplace=False)
  (4): ReLU()
  (5): Linear(in_features=32, out_features=64, bias=True)
  (6): ReLU()
  (7): Linear(in_features=64, out_features=16, bias=True)
  (8): Dropout(p=0.5, inplace=False)
  (9): ReLU()
  (10): Linear(in_features=16, out_features=1, bias=True)
  (11): Sigmoid()
)

In [28]:
optimizer = torch.optim.Adam(model.parameters(), lr=config["train"]["lr"])

In [29]:
model.to(device)
print(device)

cuda:0


In [30]:
torch.cuda.is_available()

True

In [31]:
datasetTrain = torch.utils.data.TensorDataset(trainX, trainy)
loaderTrain = torch.utils.data.DataLoader(
    datasetTrain,
    batch_size=config['train']['batchSize'],
    shuffle=True
)

datasetTest = torch.utils.data.TensorDataset(testX, testy)
loaderTest = torch.utils.data.DataLoader(
    datasetTest,
    batch_size=config['train']['batchSize'],
    shuffle=True
)

In [32]:
n_epochs = config['train']['epochs']

# disabled for now
for n in range(0):
    model.train()
    accuracy_sum = 0
    loss_sum = 0

    for (x,y) in tqdm(loaderTrain):
        y_pred = model(x)
        loss = compute_loss(y_pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        loss_sum = loss_sum + loss.cpu().detach().item()
        accuracy_sum = accuracy_sum + accuracy_fn(y_true=y, y_pred=torch.round(y_pred))

    train_accuracy = accuracy_sum / len(loaderTrain)
    train_loss = loss_sum / len(loaderTrain)

    with torch.no_grad():
        accuracy_sum = 0
        loss_sum = 0

        for (x,y) in tqdm(loaderTest):
            y_pred = model(x)
            val_loss = compute_loss(y_pred, y)

            loss_sum = loss_sum + val_loss.cpu().detach().item()
            accuracy_sum = accuracy_sum + accuracy_fn(y_true=y, y_pred=torch.round(y_pred))

    test_accuracy = accuracy_sum / len(loaderTest)
    train_loss = loss_sum / len(loaderTest)


    print(f'Epocha: {n}')
    print(f'Test accuracy: {test_accuracy} - Test Loss: {val_loss} |<->| Train Accuracy: {train_accuracy} - Train Loss: {train_loss}')
    wandb.log({'epoch': n, 'test_accuracy': test_accuracy, 'loss_val': val_loss, 'train_accuracy': train_accuracy, 'loss_train': train_loss})
#print(loss)

In [33]:
wandb.finish()

## Pytorch hyperparameter tuning code

This part contains duplicated code from above, used for hyperparameter tuning of a model

In [47]:
def pytorch_training(config : dict, model):
    """Hyperparameter tuning of pytorch models."""
    run_name = f"{config['model_name']}-lr{config['train']['lr']}-ep{config['train']['epochs']}-bs{config['train']['batchSize']}"
    wandb.init(
            project="lol-predictor",
            config=config['train'],
            group='pytorch',
            mode='online',
            name=run_name
        )
 
    optimizer = torch.optim.Adam(model.parameters(), lr=config["train"]["lr"])
    
    trainX, trainy = prepare_pytorch_split(train)
    testX, testy = prepare_pytorch_split(test)
    valX, valy = prepare_pytorch_split(validation)
    
    model.to(device)
    datasetTrain = torch.utils.data.TensorDataset(trainX, trainy)
    loaderTrain = torch.utils.data.DataLoader(
        datasetTrain,
        batch_size=config['train']['batchSize'],
        shuffle=True
    )

    datasetTest = torch.utils.data.TensorDataset(testX, testy)
    loaderTest = torch.utils.data.DataLoader(
        datasetTest,
        batch_size=config['train']['batchSize'],
        shuffle=True
    )
    
    best_accuracy = 0
    for n in range(config['train']['epochs']):
        model.train()
        accuracy_sum = 0
        loss_sum = 0

        for (x,y) in tqdm(loaderTrain):
            y_pred = model(x)
            loss = compute_loss(y_pred, y)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            loss_sum = loss_sum + loss.cpu().detach().item()
            accuracy_sum = accuracy_sum + accuracy_fn(y_true=y, y_pred=torch.round(y_pred))

        train_accuracy = accuracy_sum / len(loaderTrain)
        train_loss = loss_sum / len(loaderTrain)

        with torch.no_grad():
            accuracy_sum = 0
            loss_sum = 0
            recall_sum = 0
            precision_sum = 0

            for (x,y) in tqdm(loaderTest):                
                y_pred = model(x)
                val_loss = compute_loss(y_pred, y)

                loss_sum = loss_sum + val_loss.cpu().detach().item()
                accuracy_sum = accuracy_sum + accuracy_fn(y_true=y, y_pred=torch.round(y_pred))

                cpu_y = torch.round(y).cpu().detach().numpy()
                cpu_pred_y = torch.round(y_pred).cpu().detach().numpy()

                recall_sum += recall_score(y_true=cpu_y, y_pred=cpu_pred_y)
                precision_sum += precision_score(cpu_y, y_pred=cpu_pred_y)

        test_accuracy = accuracy_sum / len(loaderTest)
        train_loss = loss_sum / len(loaderTest)
        recall = recall_sum / len(loaderTest)
        precision = precision_sum / len(loaderTest)

        wandb.log({'epoch': n,
                   'test_accuracy': test_accuracy, 
                   'test_loss': val_loss, 
                   'train_accuracy': train_accuracy, 
                   'train_loss': train_loss,
                   'test_precision': precision,
                   'test_recall': recall
                   })
        if best_accuracy < test_accuracy:
            best_accuracy = test_accuracy
    
    np_valy = valy.cpu().detach().numpy()
    np_predy = torch.round(model(valX)).cpu().detach().numpy()
    logging.info(f"Validation metrics: \n    Accuracy: {round(accuracy_score(np_valy, np_predy), 4)}\n\
    Recall: {round(recall_score(np_valy, np_predy), 4)}\n    Precision: {round(precision_score(np_valy, np_predy), 4)}.")
    return (best_accuracy, test_accuracy)

In [48]:
def get_model(name : str):
    """Get empty model to train based on your choice"""
    nHidden = config['train']['inputNeurons']
    
    if name == 'baseline':
        return nn.Sequential(
            nn.Linear(config["preparation"]["features_amount"], nHidden[0]),
            nn.ReLU(),
            nn.Linear(nHidden[0], nHidden[1]),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(nHidden[1], nHidden[2]),
            nn.ReLU(),
            nn.Linear(nHidden[2], nHidden[3]),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(nHidden[3], 1),
            nn.Sigmoid()
            # Softmax
        )
    
    if name == 'small':
        return nn.Sequential(
            nn.Linear(config["preparation"]["features_amount"], 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    if name == 'nano':
        return nn.Sequential(
            nn.Linear(config["preparation"]["features_amount"], 16),
            nn.ReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    
    if name == 'LeakyNano':
        return nn.Sequential(
            nn.Linear(config["preparation"]["features_amount"], 16),
            nn.LeakyReLU(),
            nn.Linear(16, 1),
            nn.Sigmoid()
        )
    

In [49]:
hyp_search = {'lr': [0.01, 0.005, 0.001, 0.0005],
              'epochs': [25, 50, 100, 200],
              'batchSize': [256, 512, 99999],
              'model_names': ['baseline', 'small', 'nano', 'LeakyNano']
             }

In [45]:
"""
hyp_search = {'lr': [0.01],
              'epochs': [200],
              'batchSize': [6385],
              'model_names': ['baseline']
            }
"""

"\nhyp_search = {'lr': [0.01],\n              'epochs': [200],\n              'batchSize': [6385],\n              'model_names': ['baseline']\n            }\n"

Test different parameters and model architectures and evaluate on validation set with accuracy, recall and precission score as result.\
Higher number in all department is better, with scale being 0-1.

In [50]:
%%capture --no-stdout
# silence everything, but prints from this cell
# 53m 28sec

test_conf = {'train' : {}}

for testing_model in hyp_search["model_names"]:
    test_conf['model_name'] = testing_model
    for learn_rate in hyp_search['lr']:
        test_conf['train']['lr'] = learn_rate

        for epochs in hyp_search['epochs']:
            test_conf['train']['epochs'] = epochs

            for batchSize in hyp_search['batchSize']:
                test_conf['train']['batchSize'] = batchSize
                logging.info(f"Training - {testing_model}: {test_conf['train']}")
                best_acc, final_acc = pytorch_training(test_conf, get_model(testing_model))
                logging.info(f"Results - Best accuracy on test: {round(best_acc, 4)}, finished on {round(final_acc, 4)}")

## Tensorflow implementation

In [37]:
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import tempfile
import os

from wandb.keras import WandbMetricsLogger
import tensorflow as tf
from tensorflow import keras

In [38]:
wandb.login()

wandb.init(
        project="lol-predictor",
        config=config['train'],
        group='tensorflow',
        mode='online'
    )

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


batch/accuracy,▁▅▅▅▅▅▅▅▅▅▅▆▅█▅▅▇▅▅▅▆▆▆▇▆▆▆▆▆▃▆▆▇▆▆▃▆▆▆▇
batch/batch_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
batch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,█▄▄▄▄▄▄▄▄▄▄▄▄▃▄▄▁▄▄▄▄▄▃▃▃▃▃▃▃▅▃▃▂▃▃▅▃▂▂▂
epoch/accuracy,▁▃▃▄▄▄▄▄▄▄▄▄▄▄▄▅▅▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
epoch/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▆▆▆▅▅▅▅▅▅▅▅▅▅▅▅▅▄▅▄▄▄▄▄▄▄▄▃▃▃▃▃▃▂▂▂▂▂▁▁
epoch/val_accuracy,▃███▅▇▇▇▅▆▆▅▄▅▅▆▆▅▆▅▅▃▆▃▄▄▄▄▄▃▃▂▂▂▃▁▂▃▁▁
epoch/val_loss,▃▁▁▁▂▁▁▁▂▁▁▂▂▂▂▂▂▂▂▂▂▃▂▃▃▃▃▄▃▄▄▆▄▇▅▇▇▇██
batch/accuracy,0.75842


In [39]:
nHidden = config['train']['inputNeurons']

tf_model = keras.Sequential([
    keras.layers.Dense(units=20, activation='relu'),
    keras.layers.Dense(units=nHidden[0], activation='relu'),
    keras.layers.Dense(units=nHidden[1], activation='relu'),
    keras.layers.Dense(units=nHidden[2], activation='relu'),
    keras.layers.Dense(units=nHidden[3], activation='relu'),  #activation='softmax'
    keras.layers.Dense(units=1, activation='sigmoid')
])

tf_model.compile(optimizer='adam', 
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [40]:
trainX = train.drop(columns={"blueWins"})
trainy = train['blueWins']
testX = test.drop(columns={"blueWins"})
testy = test['blueWins']

In [41]:
tf_model.fit(trainX, trainy, validation_data=(testX, testy), epochs=50, batch_size=32, callbacks=[WandbMetricsLogger(log_freq=32)])

Epoch 1/50


/home/xschon/.local/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


200/200 [==============================] - 1s 2ms/step - loss: 0.6062 - accuracy: 0.6684 - val_loss: 0.5506 - val_accuracy: 0.7267
Epoch 2/50
 36/200 [====>.........................] - ETA: 0s - loss: 0.5420 - accuracy: 0.7274

/home/xschon/.local/lib/python3.10/site-packages/keras/backend.py:5676: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


200/200 [==============================] - 0s 2ms/step - loss: 0.5583 - accuracy: 0.7142 - val_loss: 0.5581 - val_accuracy: 0.7173
Epoch 3/50
200/200 [==============================] - 0s 2ms/step - loss: 0.5528 - accuracy: 0.7156 - val_loss: 0.5490 - val_accuracy: 0.7301
Epoch 4/50
200/200 [==============================] - 0s 2ms/step - loss: 0.5511 - accuracy: 0.7145 - val_loss: 0.5460 - val_accuracy: 0.7321
Epoch 5/50
200/200 [==============================] - 0s 2ms/step - loss: 0.5484 - accuracy: 0.7206 - val_loss: 0.5514 - val_accuracy: 0.7294
Epoch 6/50
200/200 [==============================] - 0s 2ms/step - loss: 0.5476 - accuracy: 0.7211 - val_loss: 0.5771 - val_accuracy: 0.7031
Epoch 7/50
200/200 [==============================] - 0s 2ms/step - loss: 0.5504 - accuracy: 0.7181 - val_loss: 0.5450 - val_accuracy: 0.7308
Epoch 8/50
200/200 [==============================] - 0s 2ms/step - loss: 0.5454 - accuracy: 0.7206 - val_loss: 0.5495 - val_accuracy: 0.7247
Epoch 9/50
200/20